In [ ]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

symbols = ['A', 'B', 'C', 'D', 'W', 'S']
frequencies = {'A': 20, 'B': 20, 'C': 15, 'D': 10, 'W': 5, 'S': 5}

def build_reel():
    reel = []
    for symbol, freq in frequencies.items():
        reel.extend([symbol] * freq)
    random.shuffle(reel)
    return reel

paylines = [
    [1, 1, 1, 1, 1],
    [0, 0, 0, 0, 0],  
    [2, 2, 2, 2, 2],  
    [0, 1, 2, 1, 0],  
    [2, 1, 0, 1, 2]   
]

payout_table = {
    'A': [0, 0, 5, 10, 25],
    'B': [0, 0, 5, 15, 30],
    'C': [0, 0, 10, 25, 50],
    'D': [0, 0, 20, 50, 100],
}

def simulate_spin(reels):
    grid = [[random.choice(reel) for reel in reels] for _ in range(3)]
    return grid

def evaluate_spin(grid, paylines, payout_table):
    total_win = 0
    for line in paylines:
        line_symbols = [grid[row][col] for col, row in enumerate(line)]
        base_symbol = line_symbols[0]
        if base_symbol not in payout_table:
            continue
        match_count = 1
        for sym in line_symbols[1:]:
            if sym == base_symbol or sym == 'W':
                match_count += 1
            else:
                break
        if match_count >= 3:
            index = match_count - 1
            total_win += payout_table[base_symbol][index]
    return total_win

def simulate_bonus_round():
    states = ['start', 'spin', 'end']
    transition_matrix = {
        'start': {'spin': 1.0},
        'spin': {'spin': 0.6, 'end': 0.4}
    }
    current_state = 'start'
    bonus_total = 0
    while current_state != 'end':
        next_probs = transition_matrix[current_state]
        states = list(next_probs.keys())
        weights = list(next_probs.values())
        next_state = random.choices(states, weights)[0]     
        if next_state == 'spin':
            reward = random.choice([5, 10, 20, 0, 0])
            bonus_total += reward
        current_state = next_state
    return bonus_total

num_spins = 10000
reels = [build_reel() for _ in range(5)]
results = []
for _ in range(num_spins):
    grid = simulate_spin(reels)
    base_win = evaluate_spin(grid, paylines, payout_table)
    bonus_win = simulate_bonus_round() if random.random() < 0.1 else 0
    total_win = base_win + bonus_win
    results.append(total_win)

df = pd.DataFrame({'Win': results})

plt.figure(figsize=(10, 5))
plt.hist(df['Win'], bins=range(0, max(df['Win']) + 5, 5), color='skyblue', edgecolor='black')
plt.title("Win amounts over 10,000 spins")
plt.xlabel("Win amount")
plt.ylabel("Frequency")
plt.grid(True)
plt.tight_layout()
plt.show()

volatility = df['Win'].std()
kurtosis = df['Win'].kurtosis()
print(f"Volatility as std dev of wins): {volatility:.2f}")
print(f"Kurtosis of win distribution: {kurtosis:.2f}")

num_trials = 10
all_rtp_curves = []
colors = plt.cm.tab10(np.linspace(0, 1, num_trials))

for _ in range(num_trials):
    reels = [build_reel() for _ in range(5)]
    results = []
    for _ in range(num_spins):
        grid = simulate_spin(reels)
        base_win = evaluate_spin(grid, paylines, payout_table)
        bonus_win = simulate_bonus_round() if random.random() < 0.1 else 0
        total_win = base_win + bonus_win
        results.append(total_win)
    df_temp = pd.DataFrame({'Win': results})
    rtp_curve = df_temp['Win'].cumsum() / ((df_temp.index + 1) * 1)
    all_rtp_curves.append(rtp_curve)

plt.figure(figsize=(12, 6))
for i, curve in enumerate(all_rtp_curves):
    plt.plot(curve, color=colors[i], linewidth=1.5, alpha=0.7)
plt.title("Cumulative RTP over 10 simulations of 10000 spins")
plt.xlabel("Number of spins")
plt.ylabel("Cumulative RTP")
plt.grid(True)
plt.tight_layout()
plt.show()
